In [3]:
import pandas as pd
import sys
sys.path.append("..")
import utility.utility_functions as ut

In [18]:
week_end = ['Saturday','Sunday']
mid_week = ['Monday','Tuesday','Wednesday','Thursday','Friday']

In [12]:
midnight = [
            '00:00 - 00:59',
            '01:00 - 01:59',
            '02:00 - 02:59',
            '03:00 - 03:59',
            '04:00 - 04:59',
            '05:00 - 05:59',
            '06:00 - 06:59'
            ]
morning = [
            '07:00 - 07:59',
            '08:00 - 08:59',
            '09:00 - 09:59',
            '10:00 - 10:59',
            '11:00 - 11:59'
            ]
noon_afternoon = [
            '12:00 - 12:59',
            '13:00 - 13:59',
            '14:00 - 14:59',
            '15:00 - 15:59',
            '16:00 - 16:59',
            '17:00 - 17:59',
            '18:00 - 18:59',
            '19:00 - 19:59'
            ]
night = [
            '20:00 - 20:59',
            '21:00 - 21:59',
            '22:00 - 22:59',
            '23:00 - 23:59'
]

In [4]:
df = ut.load_post_data_prep_all_zone()
df.dropna(inplace=True)
df = df.drop(df[df.trip_time < 5].index)
df = df.drop('pickup_date',axis=1)
df.reset_index(inplace=True,drop=True)

In [7]:
main_df = ut.swap_first_and_last_col(df)

In [22]:
mid_week_morning_df = main_df.loc[(main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(morning))]
mid_week_night_df = main_df.loc[(main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(night))]
mid_week_noon_afternoon_df = main_df.loc[(main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(noon_afternoon))]
mid_week_midnight_df = main_df.loc[(main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(midnight))]


In [23]:
week_end_morning_df = main_df.loc[(main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(morning))]
week_end_night_df = main_df.loc[(main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(night))]
week_end_noon_afternoon_df = main_df.loc[(main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(noon_afternoon))]
week_end_midnight_df = main_df.loc[(main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(midnight))]